In [1]:
import pandas as pd
import procesing_pdf as ppdf
from embedding_model import extract_features
import re
import os
import numpy as np


/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/renato/miniforge3

101,4450,3006,28215,1998,1996,5157,2005,3119,3860,3350,2013,3784,12265,10999,4487,2425,2050,1998,19522,8473,15564,1050,5677,2551,3259,2053,2233,15333,2140,2053,6904,5910,6494,6593,2057,2817,18394,2005,2231,2895,1999,3433,2000,3913,27475,4525,2013,2367,4127,1997,4450,20285,28215,2057,5136,1996,2206,28215,10660,2689,1037,5157,5670,2919,2968,1998,2093,7957,1997,2248,21100,8162,6129,25094,2024,2445,1037,3601,2426,2053,2231,2895,4012,11837,16846,10253,15210,1998,3119,3860,1999,3433,2000,2122,28215,2490,2005,2231,8830,3227,9466,9249,1998,2003,4600,25352,2875,3119,3860,5157,2005,12324,3860,7457,6022,1999,2035,3572,3272,2005,1996,2919,2968,5213,3119,28215,9699,2062,5157,2005,3860,2964,1998,2426,3119,28215,21100,8162,6129,2000,1037,4975,2406,12005,26243,2015,3618,5157,2005,3860,2964,2084,21100,8162,6129,2000,1037,2764,2406,1996,2919,2968,5213,2003,1996,2069,11967,2008,19653,2015,1037,9059,3623,1999,4012,11837,16846,10253,15210,3896,3711,2000,2022,21770,10624,6914,14769,2408,20576,2015,2007,2367

In [2]:
df1=pd.read_excel("train_base.xlsx",index_col=False)
df2=pd.read_excel("train_base_2.xlsx",index_col=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_base_2.xlsx'

In [39]:
columns_to_concatenate = [7] + list(range(19, 31))

# Extraer las columnas de ambos DataFrames
df1_selected = df1.iloc[:, columns_to_concatenate]
df2_selected = df2.iloc[:, columns_to_concatenate]

# Concatenar las filas de df2 debajo de df1
result = pd.concat([df1_selected, df2_selected], ignore_index=True)
result

,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,Facilitating learning and discovery-oriented i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes
1,Responding Adaptively to the COVID-19 Crisis i...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes,NaN,NaN,NaN
2,Vaccines for a sustainable planet,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,Estimates and Projections of the Global Econom...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,Cohort Profile: Real-Time Insights of COVID-19...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"Aspirations, Social Norms and Development",NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN
241,"Entertainment, Education and Attitudes towards...",NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN
242,How ed-tech can help leapfrog progress in educ...,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
243,Behavioral Inattention,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
result.to_excel("train_base_concatenated.xlsx", index=False)

NameError: name 'result' is not defined

In [2]:
def replace_colons_with_underscores(text):
    invalid_characters = r'[<>:"/\\|?*]'
    return re.sub(invalid_characters, '_', text)
download_dir="/home/renato/Proyects/Work/scraping/pdf_train"

In [3]:
mapping = {
    'Yes': 1,
    np.nan: 0
}
df=pd.read_excel("train_base_concatenated.xlsx",index_col=False)
cols_to_replace = df.columns[-12:]
df[cols_to_replace] = df[cols_to_replace].applymap(lambda x: mapping.get(x, 0)) # type: ignore
df

/tmp/ipykernel_37393/2365230671.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_to_replace] = df[cols_to_replace].applymap(lambda x: mapping.get(x, 0)) # type: ignore


,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,Facilitating learning and discovery-oriented i...,0,0,0,0,0,0,0,0,1,0,0,1
1,Responding Adaptively to the COVID-19 Crisis i...,0,0,0,0,0,1,0,0,1,0,0,0
2,Vaccines for a sustainable planet,0,0,0,1,0,1,0,0,0,0,0,0
3,Estimates and Projections of the Global Econom...,0,0,0,0,0,1,0,0,0,0,0,0
4,Cohort Profile: Real-Time Insights of COVID-19...,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"Aspirations, Social Norms and Development",0,0,1,0,0,0,1,0,1,0,0,0
241,"Entertainment, Education and Attitudes towards...",0,0,0,0,1,0,1,0,0,0,0,0
242,How ed-tech can help leapfrog progress in educ...,0,0,1,0,0,0,0,0,0,0,1,0
243,Behavioral Inattention,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
rows_to_keep = []
for index, row in df.iterrows():
    pdf_filename = f"{row['Title']}.pdf"
    pdf_filename=replace_colons_with_underscores(pdf_filename)
    pdf_path = os.path.join(download_dir, pdf_filename)
    if os.path.exists(pdf_path):
        text=ppdf.get_clean_text(replace_colons_with_underscores(row['Title']))
        embedings=extract_features(text).numpy()
        tensor_string = ','.join(map(str,embedings))
        df.at[index, 'Title'] = tensor_string
        rows_to_keep.append(index)
    else:
        pass
df = df.loc[rows_to_keep]
df


,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,"101,25505,4083,1998,5456,10050,14088,3919,3343...",0,0,0,0,0,0,0,0,1,0,0,1
1,"101,14120,19293,2135,2000,1996,2522,17258,2677...",0,0,0,0,0,1,0,0,1,0,0,0
4,"101,3931,1997,25163,2099,16963,3802,2632,1038,...",0,0,0,0,0,1,0,0,0,0,0,0
5,"101,24479,2349,2000,2037,3020,2572,2099,4022,3...",0,0,0,0,0,1,0,0,0,0,0,0
6,"101,8814,3490,14028,3012,1997,2662,2624,5277,2...",0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"101,3485,1997,1996,2647,3171,2523,21146,30510,...",0,0,1,0,0,0,1,0,1,0,0,0
241,"101,2089,29347,2050,4981,1998,8931,10220,2000,...",0,0,0,0,1,0,1,0,0,0,0,0
242,"101,2129,3968,15007,2064,2393,11679,19699,8649...",0,0,1,0,0,0,0,0,0,0,1,0
243,"101,4127,1997,4748,14945,2521,7113,2638,3802,2...",0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df.to_excel("train_tensors.xlsx", index=False)